# Example Dashboard

Demo using `voici`.

*Currently BROKEN in `voici` dashboard view? The widgets etc. seem to work if I open the source notebook in JupyterLab, but in the `voici` view, the chart is not updated if I change the widget menu options.*

In [1]:
# Load in required packages
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Load in the data from the Dakar live data site
from dakar_rallydj.getter import DakarAPIClient

dakar = DakarAPIClient(
    use_cache=True,
    backend='memory',
    # cache_name='dakar_cache',
    expire_after=3600  # Cache for 1 hour
)

In [2]:
stages_df, sectors_df, stage_surfaces_df, section_surfaces_df, surfaces_df = dakar.get_stages()

# Get distances from percentages
stage_surfaces_df = pd.merge(
    sectors_df[["code", "length"]], stage_surfaces_df, on="code")
stage_surfaces_df["distance"] = stage_surfaces_df["length"] * \
    stage_surfaces_df["percentage"]/100

# Rename the 0P000 stage to 00000 to make a simpler sort...
#sectors_df.loc[sectors_df['code'].str.startswith('0P'), 'stage_code'] = '00000'

#sectors_df.sort_values(["stage_code", "sector_number"], inplace=True)
#sectors_df.reset_index(drop=True, inplace=True)

In [3]:
from ipywidgets import interact, widgets

plt.figure(figsize=(10, 6))

# Mapping types to colors
type_color_map = dict(
    zip(stage_surfaces_df['type'], stage_surfaces_df['color']))

def plot_stage_surface_chart(selected_code, typ):
    plt.clf()

    # Filter the DataFrame for the selected code
    filtered_df = stage_surfaces_df[stage_surfaces_df['code'] == selected_code]

    # Create the plot (remove plt.figure() call)
    sns.barplot(
        data=filtered_df,
        x="type",
        y=typ,
        palette=type_color_map,
        hue="type",
        legend=False
    )

    # Adjust labels and title
    plt.title(f"Dodged Bar Chart for Code {selected_code}")
    plt.xlabel("Surface type")
    plt.ylabel(f"{typ.capitalize()}")
    plt.xticks(rotation=45)
    plt.show()
    plt.close()


# Create dropdown widget
code_dropdown = widgets.Dropdown(
    options=stage_surfaces_df['code'].unique(),
    description='Code:',
    value=stage_surfaces_df['code'].unique()[0]
)

typ_dropdown = widgets.Dropdown(
    options=['percentage', 'distance'],  # Add options for Y-axis
    description='Y-axis:',
    value='percentage')

# Interactive plot
interact(plot_stage_surface_chart,
         selected_code=code_dropdown, typ=typ_dropdown);

<Figure size 1000x600 with 0 Axes>

interactive(children=(Dropdown(description='Code:', options=('01200', '02200', '03200', '04100', '04200', '051…